This notebook is intended for debugging the topology selection problem in OR-tools 

In [1]:
import sys
from ortools.linear_solver import pywraplp
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import random

np.set_printoptions(threshold=sys.maxsize)

To Do: 
- Fix printing 
- Figure out how to save variable solutions in a 3D array

Two matrices print at the end of the notebook. The first matrix is the flow values and the second matrix is the topology control variables. Each row represents node i and each column represents node j s.t. i!=j. For example, if n=4, then the output matrix is: 

[x12 x13 x14]<br>
[x21 x23 x24]<br>
[x31 x32 x34]<br>
[x41 x42 x43]<br>

In [12]:
'''
Helper functions -- for readability
'''

def getBidirectionalCapacityConstraints(combos, flow_var_dict, topology_var_dict, capacity, solver, infinity):
    '''
    Define the topology control and flow variables 
    and add bidirectional and flow constraints to solver
    '''

    for combo in combos:
        i, j = combo
        #flow keys
        key0 = 'x' + str(i) +'_'+ str(j) #i,j
        key1 = 'x' + str(j) +'_'+str(i) #j,i

        #add flow variables
        flow_var_dict[key0] = solver.IntVar(0.0, infinity, "x" + str(i)+str(j))
        flow_var_dict[key1] = solver.IntVar(0.0, infinity, "x" + str(j)+str(i))

        #add topology variables 
        key2 = "z" + str(i)+'_'+str(j) #i,j
        key3 = "z" + str(j)+'_'+str(i) #j,i

        topology_var_dict[key2] = solver.IntVar(0.0, 1.0, "z" + str(i)+str(j))
        topology_var_dict[key3] = solver.IntVar(0.0, 1.0, "z" + str(j)+str(i))

        #add bidriectional and capacity constraints 
        solver.Add(topology_var_dict[key2] == topology_var_dict[key3]) #bidirectional constraint

        solver.Add(flow_var_dict[key0] <= capacity[i-1,j-1]*topology_var_dict[key2]) #capacity constraint i,j
        solver.Add(flow_var_dict[key1] <= capacity[i-1,j-1]*topology_var_dict[key3]) #capacity constraint j,i

    print('Number of variables =', solver.NumVariables())  

    return 



def getFlowTopologyConstraints(flow_var_dict, topology_var_dict, rho, L_k, solver, T, source, dest):
    '''
    Define flow conservation constraint equations 
    and topology control constraints
    '''

    #flow conservation constraint equations and topology control constraint
    for i in range(1,num_nodes+1): 
        add = []
        sub = []
        add_top = [] 
        for j in range(1,num_nodes+1): 
            if i!=j: 
                key1 = 'x'+ str(i) + '_' + str(j)
                key2 = 'x' + str(j) + '_' + str(i)

                key3 = 'z' + str(i) +'_'+ str(j)

                add.append(flow_var_dict[key1]) #x_ij
                sub.append(flow_var_dict[key2]) #x_ji

                add_top.append(topology_var_dict[key3]) #z_ij

            else: 
                continue

        solver.Add(sum(add_top) <= T) #topology control

        #flow conservation piecewise function
        if type(source)!= int:
            if i in source: 
                solver.Add(sum(add)-sum(sub) == rho*L_k)
            elif i in dest: 
                solver.Add(sum(add)-sum(sub) == -rho*L_k)
            else: 
                solver.Add(sum(add)-sum(sub) == 0)
        else: 
            if i == source: 
                solver.Add(sum(add)-sum(sub) == rho*L_k)
            elif i == dest: 
                solver.Add(sum(add)-sum(sub) == -rho*L_k)
            else: 
                solver.Add(sum(add)-sum(sub) == 0)

    return

def printSolutionValues(num_nodes, flow_var_dict, topology_var_dict, rho):
    print('Solution:')
    print('p = ', rho.solution_value())

    #get solution value of variables and convert to arrays 
    flow_vals = [x.solution_value() for x in list(flow_var_dict.values())]

    flow_val_keys = list(flow_var_dict.keys()) 

    #seperate so we can aggreagate by i nodes
    evens = flow_vals[0::2]
    odds = flow_vals[1::2]

    evens_keys = flow_val_keys[0::2]
    odds_keys = flow_val_keys[1::2]

    flow_vals = evens + odds
    flow_vals_keys = evens_keys + odds_keys 

    flow_vals = np.reshape(flow_vals, (num_nodes, num_nodes-1))
    flow_vals_keys = np.reshape(flow_vals_keys, (num_nodes, num_nodes-1))

    print('flow vals')
    print(flow_vals) #each row represents a node
    
    print(flow_vals_keys)

    print('-----------------------------------------------------\n')

    topology_vals = [x.solution_value() for x in list(topology_var_dict.values())]
    topology_val_keys = list(topology_var_dict.keys()) 

    #seperate so we can aggreagate by i nodes
    evens = topology_vals[0::2]
    odds = topology_vals[1::2]

    evens_keys = topology_val_keys[0::2]
    odds_keys = topology_val_keys[1::2]

    topology_vals = evens + odds
    topology_vals_keys = evens_keys + odds_keys 

    topology_vals = np.reshape(topology_vals, (num_nodes, num_nodes-1))
    topology_vals_keys = np.reshape(topology_vals_keys, (num_nodes, num_nodes-1))

    print('topology vals')
    print(topology_vals) #each row represents a node
    
    print(topology_vals_keys)
        
    return

In [22]:
def single_source_single_dest(num_nodes, source, dest, L_k, T, capacity):
    
    '''
    Solve single commodity flow problem
    '''
    
    solver = pywraplp.Solver.CreateSolver('SAT')
    #if not solver:
        #return

    infinity = solver.infinity()

    #add rho variable
    rho = solver.NumVar(0.0, infinity, 'p')

    #define flow variables and topology control variables 
    combos = itertools.combinations(range(1,num_nodes+1), 2)
    flow_var_dict = {} 
    topology_var_dict = {} 
 
    #setup the problem
    getBidirectionalCapacityConstraints(combos, flow_var_dict, topology_var_dict, capacity, solver, infinity)
    getFlowTopologyConstraints(flow_var_dict, topology_var_dict, rho, L_k, solver, T, source, dest)
         
    #define the objective
    solver.Maximize(rho)

    #call the solver
    print(f'Solving with {solver.SolverVersion()}')
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        
        printSolutionValues(num_nodes, flow_var_dict, topology_var_dict, rho)

    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

In [39]:
source = 3 #source node
dest = 4 #destination node 
L_k = 10 #commodity, k =1 
T = 2 #number of link connections
num_nodes = 4

#fake capacity matrix 
capacity = np.random.randint(low=1, high=L_k+5, size =(num_nodes,num_nodes))
capacity = np.triu(capacity)
np.fill_diagonal(capacity, 0)

single_source_single_dest(num_nodes, source, dest, L_k, T, capacity)

Number of variables = 25
Solving with CP-SAT solver v9.6.2534
Solution:
p =  1.9
flow vals
[[ 1.  0.  7.]
 [ 0.  0. 13.]
 [ 7.  0.  1.]
 [ 6.  0.  0.]]
[['x1_2' 'x1_3' 'x1_4']
 ['x2_3' 'x2_4' 'x3_4']
 ['x2_1' 'x3_1' 'x4_1']
 ['x3_2' 'x4_2' 'x4_3']]
-----------------------------------------------------

topology vals
[[1. 0. 1.]
 [1. 0. 1.]
 [1. 0. 1.]
 [1. 0. 1.]]
[['z1_2' 'z1_3' 'z1_4']
 ['z2_3' 'z2_4' 'z3_4']
 ['z2_1' 'z3_1' 'z4_1']
 ['z3_2' 'z4_2' 'z4_3']]

Advanced usage:
Problem solved in 35.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes


### Solve through time
Still considering a single source and single destination problem, but capacity and commodity varies with time. 

In [38]:
source = 1 #source node
dest = 10 #destination node 
T = 2 #number of link connections
num_nodes = 10
t = 3 #timesteps 

L_k = np.random.randint(low=1, high=10, size=(t,))

#fake capacity matrix 
capacity = np.random.randint(low=1, high=10, size =(t, num_nodes,num_nodes))
for i in range(0,t):
    capacity[i, :, :] = np.triu(capacity[i, :, :])
    np.fill_diagonal(capacity[i, :, :], 0)

for i in range(0,t): 
    temp = capacity[i, :, :]
    print('commodity =', L_k[i])
    single_source_single_dest(num_nodes, source, dest, L_k[i], T, temp)


commodity = 8
Number of variables = 181
Solving with CP-SAT solver v9.6.2534
Solution:
p =  1.875
flow vals
[[0. 0. 0. 8. 0. 0. 0. 0. 7.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2.]
 [0. 0. 0. 0. 0. 8. 0. 0. 0.]
 [0. 0. 8. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [8. 0. 0. 0. 0. 0. 0. 0. 0.]]
[['x1_2' 'x1_3' 'x1_4' 'x1_5' 'x1_6' 'x1_7' 'x1_8' 'x1_9' 'x1_10']
 ['x2_3' 'x2_4' 'x2_5' 'x2_6' 'x2_7' 'x2_8' 'x2_9' 'x2_10' 'x3_4']
 ['x3_5' 'x3_6' 'x3_7' 'x3_8' 'x3_9' 'x3_10' 'x4_5' 'x4_6' 'x4_7']
 ['x4_8' 'x4_9' 'x4_10' 'x5_6' 'x5_7' 'x5_8' 'x5_9' 'x5_10' 'x6_7']
 ['x6_8' 'x6_9' 'x6_10' 'x7_8' 'x7_9' 'x7_10' 'x8_9' 'x8_10' 'x9_10']
 ['x2_1' 'x3_1' 'x4_1' 'x5_1' 'x6_1' 'x7_1' 'x8_1' 'x9_1' 'x10_1']
 ['x3_2' 'x4_2' 'x5_2' 'x6_2' 'x7_2' 'x8_2' 'x9_2' 'x10_2' 'x4_3']
 ['x5_3' 'x6_3' 'x7_3' 'x8_3' 'x9_3' 'x10_3' 'x5_4' 'x6_4' 'x7_4']
 ['x8_4' 'x9_4' 'x10_4' 'x6_5' 'x7_5' 'x8_5' 'x9_5'

dummy examples works through time &check;